# AVI Demo Walkthrough

https://github.com/vmware/alb-sdk/blob/22.1.3/python/avi/sdk


<img src="files/imagen2.png">

### A Basic VS require some objects to be created beforehand

       VSVIP.- IP Address used to expose the service. (Can be static or autoallocated by IPAM)
       POOL.- A set of Servers forming a common farm with same contents
       SE_GROUP.- (default = Default-Group). Service Engine Group in where the VS is going to be realized
       CLOUD.- (default = Default-Cloud). Used to specify the infra provider the object belongs to


### Import Libraries

In [ ]:
# Import libraries (pip install avisdk first to install AVI SDK)
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### Import Environment Variables

In [ ]:
from envs.demo_env import demo_env_params as demo_env

 ### Establish Session with Controller 

In [ ]:
# Establish a first session
api = ApiSession(
    controller_ip=demo_env['controller_ip'],
    username=demo_env['controller_username'],
    password=demo_env['controller_password'],
    tenant=demo_env['tenant'],
    api_version=demo_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
demo_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
demo_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=demo_env['controller_ip'],
    username=demo_env['controller_username'],
    password=demo_env['controller_password'],
    tenant=demo_env['tenant'],
    api_version=demo_env['api_version']
    )

### Verify controller connectivity by showing session ID 

In [ ]:
print('Successful connection to ' + demo_env['name'] + '. Session ID:' + api.session_id)

### Get Cloud and SE_Group References

In [ ]:
# Get cloud uuid (cloud = connector with any infra environment such as vcenter, nsx-t, aws, gcp... etc)
cloud = api.get_object_by_name('cloud', demo_env['cloud'])
cloud_ref = api.get_obj_ref(cloud)
print('The cloud reference is ' + cloud_ref)

In [ ]:
# Get se_group_ref  (Service Engine Group defines a common set of policies (size, HA scheme, etc) for a given set of Service Engines
se_group = api.get_object_by_name('serviceenginegroup', demo_env['service_engine_group'])
se_group_ref = api.get_obj_ref(se_group)
print('The se_group reference is ' +se_group_ref)

## Define Virtual_Service Configuration Parameters

In [ ]:
vs = {
    'vsvip_ip': "10.151.80.88",
    'name': "API",
    'port': "6443",
    'tier1_lr': "T1-DATA",
    'network_name': "AVI-DATA-OVERLAY-10.121.100.0",
    'servers': [{
        "server1": {
            "addr": "10.121.100.150",
            "name": "member_001",
            "port": "80"
            },
        "server2": {
            "addr": "10.121.100.151",
            "name": "member_002",
            "port": "80"
            },
        "server3": {
            "addr": "10.121.100.152",
            "name": "member_003",
            "port": "80"
            }
    }]       
}

### Create Pool

In [ ]:
# Create Pool Object
pool_obj = {
    "lb_algorithm": 'LB_ALGORITHM_LEAST_CONNECTIONS',
    "default_server_port": vs['port'],
    "name": "pool-"+vs['name'],
    "cloud_ref": cloud_ref,
    "servers": [{
                'hostname': vs['servers'][0]['server1']['name'],
                'ip': {
                     'addr': vs['servers'][0]['server1']['addr'], 
                     'type': 'V4'
                      },
                'port': vs['servers'][0]['server1']['port']
                },
                {
                'hostname': vs['servers'][0]['server2']['name'],
                'ip': {
                     'addr': vs['servers'][0]['server2']['addr'], 
                     'type': 'V4'
                      },
                'port': vs['servers'][0]['server2']['port']
                },
                {
                'hostname': vs['servers'][0]['server3']['name'],
                'ip': {
                     'addr': vs['servers'][0]['server3']['addr'], 
                     'type': 'V4'
                      },
                'port': vs['servers'][0]['server3']['port']
                }
        ],
        "health_monitor_refs": ['/api/healthmonitor?name=System-TCP'],
        "tier1_lr": "/infra/tier-1s/"+vs['tier1_lr']
        }

In [ ]:
resp = api.post('pool', data=json.dumps(pool_obj))

if resp.status_code in range(200, 299):
    print(resp)
    print('- Pool '+pool_obj['name'], resp.reason)#, resp.text)
else:
    print('Error in creating pool :%s' % resp.text)

### Create VS_VIP (static allocation)

In [ ]:
#vsvip_obj = {
#            'name': "vsvip-"+vs['name'],
#            'vip': [{
#                   'vip_id': '1',
#                   'ip_address':{
#                       'addr': vs['vsvip_ip'],
#                       'type': 'V4'
#                       }
#                   }]
#            }
#resp = api.post('vsvip', data=json.dumps(vsvip_obj))
#if resp.status_code in range(200, 299):
#    print(resp)
#    vsvip = api.get_object_by_name('vsvip', vsvip_obj['name'])
#    allocated_IP = vsvip['vip'][0]['ip_address']['addr']
#    print("The configured IP is: "+allocated_IP)
#else:
#    print('Error in creating vsvip :%s' % resp.text)

### Create VS_VIP (autoallocate)

In [ ]:
# Get placement network subnet
network = api.get_object_by_name('network', vs['network_name'])

# Extract subnet, mask and network_ref
subnet = network['configured_subnets'][0]['prefix']['ip_addr']['addr']
mask = network['configured_subnets'][0]['prefix']['mask']
network_ref = network['url']+"#"+network['name']

In [ ]:
# Print Extracted information
print("Network Name is: "+ network['name'])
print("Subnet is: "+subnet)
print("Mask is: "+str(mask))
print("Network_ref is: "+network_ref)

In [ ]:
vsvip_obj = {
  "cloud_ref": cloud_ref,
  "vip": [
    {
      "enabled": "true",
      "auto_allocate_ip": "true",
      "auto_allocate_ip_type": "V4_ONLY",
      "ipam_network_subnet": {
          "network_ref": network_ref,
          "subnet": {
            "ip_addr": {
               "addr": subnet,
               "type": "V4"
            },
            "mask": mask
        }
      },
      "vip_id": 1
    }
  ],
  "name": "vsVip-"+vs['name'],
  "tier1_lr": "/infra/tier-1s/"+vs['tier1_lr']
}

In [ ]:
resp = api.post('vsvip', data=json.dumps(vsvip_obj))
if resp.status_code in range(200, 299):
    print(resp)
    vsvip = api.get_object_by_name('vsvip', vsvip_obj['name'])
    allocated_IP = vsvip['vip'][0]['ip_address']['addr']
    print("The allocated IP is: "+allocated_IP)
else:
    print('Error in creating vsvip :%s' % resp.text)

### Create Virtual Service 

In [ ]:
# Gather ref of created pool and VSVIP
get_pool_obj = api.get_object_by_name('pool', pool_obj['name'])
pool_ref = api.get_obj_ref(get_pool_obj)
get_vsvip_obj = api.get_object_by_name('vsvip', vsvip_obj['name'])
vsvip_ref = api.get_obj_ref(get_vsvip_obj)

# Define services object
services_obj = [{'port': vs['port'], 'enable_ssl': False}]

# Define vs_obj
vs_obj = {
         'cloud_ref': cloud_ref,
         'name': "vs-"+vs['name'],
         'vsvip_ref': vsvip_ref,
         'services': services_obj, 
         'pool_ref': pool_ref,
         'se_group_ref': se_group_ref
         }

In [ ]:
# Call AVI virtualservice API using post and the above json as body 
resp = api.post('virtualservice', data=json.dumps(vs_obj))
if resp.status_code in range(200, 299):
    print(resp)
    print('- VirtualService '+vs_obj['name'], resp.reason)#, resp.text)
else:
    print('Error in creating vsvip :%s' % resp.text)

## Get metric of VSs

In [ ]:
# VS_Metrics
#metric_id='l4_client.avg_bandwidth'
metric_id='l4_client.max_open_conns'
entity_type='virtualservice'

resp = api.get_object_by_name(entity_type, vs_obj['name'])
uuid = api.get_obj_uuid(resp)
path = 'analytics/metrics/%s/%s/?metric_id=%s&step=300&limit=10' % (entity_type, uuid, metric_id)
resp = api.get(path)
if resp.status_code in range(200, 299):
    metrics_dict = json.loads(resp.text)
    #logger.debug('%s', metrics_dict)
    #print(json.dumps(metrics_dict, indent=2))
else:
    logger.debug('Error in getting %s metric for name : %s' % (entity_type, name))

In [ ]:
df = pd.read_json(json.dumps(metrics_dict['series'][0]['data']))
df.plot(x="timestamp", y="value")
plt.xlabel("Time", size=10)
plt.ylabel(metric_id, size=10)
plt.title("Metrics", size=20)

## Get inventory of VSs

In [ ]:
inventory_type='virtualservice-inventory'
entity_type='virtualservice'

resp = api.get_object_by_name(entity_type, vs_obj['name'])
uuid = api.get_obj_uuid(resp)

params = {}
params['page'] = 1
params['page_size'] = 10
inventory_path = inventory_type
if uuid:
    # this is case of inventory for a specific object
    inventory_path = inventory_path + '/' + uuid
resp = api.get(inventory_path, params=params)

if resp.status_code in range(200, 299):
    json.loads(resp.text)
else:
    'Error in getting inventory for %s, Error :%s' % (inventory_type, resp.text)

In [ ]:
json.loads(resp.text)